1. Réalisez un automate élémentaire.

Il faut d'abord fixer quelques règles, comme par exemple pour le cas ou une cellule atteint un bord mais qu'il reste encore des lignes a générer.
Je vais partir sur un principe de bords ouverts, donc si une cellule arrive a un bord (exemple: bord gauche), alors sont voisin de gauche sera la cellule a l'extreme droite et vice versa.
Pour éviter de se trainer une liste de liste de x*y case, je vais juste créer une liste vide qui seras la grille des générations, et à chaque génération, je vais créer une liste pour la generation actuel et l'ajouter à la liste des générations.
On aura donc 3 paramètres: la regle, la génération initial et le nombre de génération voulu.

In [12]:
def automate_elementaire(regle, initial, nb_gen):
    generations=[]
    regle_bin = [int(bit) for bit in format(regle, '08b')[::-1]]
    size = len(initial)
    generations.append(initial)
    for i in range(nb_gen):
        generation = []
        for j in range(size):
            motif = int(str(generations[i][(j-1)%size]) + str(generations[i][j]) + str(generations[i][(j+1)%size]),2)
            generation.append(regle_bin[motif])
        generations.append(generation)
    return generations

2. Vous pouvez prendre la règle d'évolution de votre choix, et faire marcher cet automate, en mode texte, dans un terminal.

In [13]:
automate = automate_elementaire(30, [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], 10)

for gen in automate:
    pimp_gen = ['🟥' if bit == 0 else '🟩' for bit in gen]
    print(pimp_gen)
    print()

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥']

['🟥', '🟥', '🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥', '🟩', '🟩', '🟩', '🟩', '🟩', '🟩', '🟥', '🟥']

['🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩', '🟥', '🟥', '🟥', '🟥', '🟥', '🟩', '🟥']

['🟩', '🟩', '🟥', '🟩', '🟩', '🟩', '🟩', '🟥', '🟩', '🟩', '🟥', '🟥', '🟩', '🟥', '🟥', '🟥', '🟩', '🟩', '🟩']

['🟥', '🟥', '🟥', '🟩', '🟥', '🟥',

3. Pour l'améliorer, on peut penser demander à l'utilisateur son choix de règle, puis en faire une représentation graphique.

Pour l'interface graphique je vais utiliser pygame.

In [17]:
import pygame
import sys
import math

# Initialisation de Pygame
pygame.init()

numero_regle = 30

def lancer_automate(num_regle, grille, nb_gen):
    initial = grille[0]
    return automate_elementaire(num_regle, initial, nb_gen)

# Fonction pour afficher la fenêtre Pygame avec la règle et les boutons
def afficher_fenetre(regle):
    global numero_regle  # Déclarer numero_regle comme une variable globale

    width, height = 800, 800
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Programme Pygame")

    # Couleurs
    white = (255, 255, 255)
    black = (0, 0, 0)
    
    # Police
    font = pygame.font.Font(None, 20)

    # Grille
    nb_colonnes = 100
    cell_size = width / nb_colonnes
    grille = [[0] * nb_colonnes for _ in range(1)]
    top = 20
    print(grille)

    # Boucle principale de la fenêtre Pygame
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                x, y = event.pos
                
                print(x, y)
                if cell_size+top < y < 2 * cell_size+top:
                    ligne = 0
                    colonne = math.floor(x / cell_size)
                    grille[ligne][colonne] = (grille[ligne][colonne] + 1) % 2
                else:
                    print("not grid")
            elif event.type == pygame.KEYDOWN:
                print(event.key)
                if event.key == pygame.K_BACKSPACE:
                    numero_regle = numero_regle // 10
                elif event.key in (pygame.K_0, pygame.K_KP0):
                    numero_regle = numero_regle * 10
                elif event.key in (pygame.K_KP1, pygame.K_1):
                    numero_regle = numero_regle * 10 + 1
                elif event.key in (pygame.K_KP2, pygame.K_2):
                    numero_regle = numero_regle * 10 + 2
                elif event.key in (pygame.K_KP3, pygame.K_3):
                    numero_regle = numero_regle * 10 + 3
                elif event.key in (pygame.K_KP4, pygame.K_4):
                    numero_regle = numero_regle * 10 + 4
                elif event.key in (pygame.K_KP5, pygame.K_5):
                    numero_regle = numero_regle * 10 + 5
                elif event.key in (pygame.K_KP6, pygame.K_6):
                    numero_regle = numero_regle * 10 + 6
                elif event.key in (pygame.K_KP7, pygame.K_7):
                    numero_regle = numero_regle * 10 + 7
                elif event.key in (pygame.K_KP8, pygame.K_8):
                    numero_regle = numero_regle * 10 + 8
                elif event.key in (pygame.K_KP9, pygame.K_9):
                    numero_regle = numero_regle * 10 + 9
                elif event.key in (pygame.K_KP_PLUS, pygame.K_PLUS):
                    numero_regle = numero_regle + 1
                elif event.key in (pygame.K_KP_MINUS, pygame.K_MINUS):
                    numero_regle = numero_regle - 1

        # Dessiner le fond
        screen.fill(white)

        grille = lancer_automate(numero_regle, grille, nb_colonnes - 1)

        # Dessiner les cellules
        for ligne in range(len(grille)):
            for colonne in range(len(grille[0])):
                couleur = black if grille[ligne][colonne] == 1 else white
                pygame.draw.rect(screen, couleur, (colonne * cell_size, (ligne + 1) * cell_size+top, cell_size, cell_size))

        # Dessiner les lignes de la grille
        for ligne in range(len(grille) + 1):
            pygame.draw.line(screen, black, (0, (ligne + 1) * cell_size+top), (width, (ligne + 1) * cell_size+top))
        # Dessiner les colonnes de la grille
        for colonne in range(len(grille[0]) + 1):
            pygame.draw.line(screen, black, (colonne * cell_size, cell_size+top), (colonne * cell_size, (len(grille) + 1) * cell_size+top))

        # Dessiner la règle
        text_regle = font.render(f"Règle : {numero_regle}", True, (0, 0, 0))
        screen.blit(text_regle, (350, cell_size / 3))

        # Mettre à jour l'affichage
        pygame.display.flip()

        # Limiter la vitesse de la boucle
        pygame.time.Clock().tick(60)

# Afficher la fenêtre Pygame avec la règle par défaut (30)
afficher_fenetre(numero_regle)


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
373 33
407 33
399 46
not grid
422 34
419 17
not grid
392 32
1073741911
1073741911
1073741911
1073741911
1073741910
1073741910
1073741910
1073741910
1073741910
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741910
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741910
1073741911
1073741911
1073741911
1073741911
1073741910
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911
1073741911

SystemExit: 

C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
